In [ ]:
%pip install pillow transformers torch huggingface_hub ipywidgets

In [ ]:
from PIL import Image
from transformers import AutoProcessor, AutoModel
import torch

device = "mps" if torch.backends.mps.is_available() else ("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModel.from_pretrained("google/medsiglip-448").to(device)
processor = AutoProcessor.from_pretrained("google/medsiglip-448")

def crop_and_resize(img, size=(448, 448)):
    w, h = img.size
    aspect = size[0] / size[1]
    img_aspect = w / h

    if img_aspect > aspect:
        # Crop width
        new_w = int(h * aspect)
        left = (w - new_w) // 2
        img = img.crop((left, 0, left + new_w, h))
    else:
        # Crop height
        new_h = int(w / aspect)
        top = (h - new_h) // 2
        img = img.crop((0, top, w, top + new_h))
    return img.resize(size, Image.BILINEAR)


In [ ]:
image_names = ["1.jpg", "2.jpg"]
imgs = [crop_and_resize(Image.open(name).convert("RGB")) for name in image_names]

texts = [
    "A fundus photograph of a healthy retina with no signs of diabetic retinopathy.",
    "A fundus photograph showing mild non-proliferative diabetic retinopathy, characterized by a few microaneurysms.",
    "A fundus photograph showing moderate non-proliferative diabetic retinopathy, with multiple microaneurysms, dot-blot hemorrhages, and cotton wool spots.",
    "A fundus photograph showing severe non-proliferative diabetic retinopathy, with extensive hemorrhages, venous beading, or intraretinal microvascular abnormalities (IRMA).",
    "A fundus photograph showing proliferative diabetic retinopathy, characterized by neovascularization (growth of new, abnormal blood vessels).",
]

short_texts = [
    "No DR",
    "Mild DR",
    "Moderate DR",
    "Severe DR",
    "PDR",
]


inputs = processor(text=texts, images=imgs, padding="max_length", return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model(**inputs)

logits_per_image = outputs.logits_per_image
probs = torch.softmax(logits_per_image, dim=1)

for n_img, img in enumerate(imgs):
    display(img)
    for i, label in enumerate(short_texts):
        print(f"{probs[n_img][i]:.2%} of '{label}'")

    max_prob, max_idx = torch.max(probs[n_img], dim=0)
    print(f"Predicted class: {short_texts[max_idx]} ({max_prob:.2%} confidence)")
    print("-" * 40)
